In [3]:
from langchain import OpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import retrieval_qa

In [4]:
loader= TextLoader('reviews.txt')
text=loader.load()

In [5]:
text=text[:500]

In [6]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=100)
new_text=text_splitter.split_documents(text)
print(new_text[0])


page_content='No issues.' metadata={'source': 'reviews.txt'}


In [12]:
os.getenv('OPENAI_API_KEY')

In [14]:
embeddings=OpenAIEmbeddings(openai_api_key=key)
db=Chroma.from_documents(new_text,embeddings,collection_name='reviews')
llm=OpenAI(openai_api_key=key,temperature=0)
chain=retrieval_qa.from_chain_type(llm,retriever=db.as_retriever())

In [ ]:
print(chain.run('how to improve the customer experience for a customer that complained about receiving his package late?'))

NameError: name 'chain' is not defined